In [1]:
import requests
import psycopg2
import pandas as pd
import numpy as np
import warnings

In [18]:
# Get data into a dataframe

api_url = "https://api.coingecko.com/api/v3/exchange_rates"
response = requests.get(api_url)

data = response.json()
rates = data['rates']

tempdf = pd.DataFrame.from_dict(rates)
df = tempdf.T

print(df)

                            name  unit        value       type
btc                      Bitcoin   BTC          1.0     crypto
eth                        Ether   ETH       13.261     crypto
ltc                     Litecoin   LTC      216.624     crypto
bch                 Bitcoin Cash   BCH      153.197     crypto
bnb                 Binance Coin   BNB       57.904     crypto
...                          ...   ...          ...        ...
xdr   IMF Special Drawing Rights   XDR    12899.191       fiat
xag          Silver - Troy Ounce   XAG       745.79  commodity
xau            Gold - Troy Ounce   XAU        9.409  commodity
bits                        Bits  μBTC    1000000.0     crypto
sats                     Satoshi  sats  100000000.0     crypto

[61 rows x 4 columns]


In [3]:
conn = psycopg2.connect(
    database="crypto",
    user="jkamino",
    password="Tacoman123",
    host="data-5330-1.ckpcm5tpab0e.us-west-1.rds.amazonaws.com",
    port="5432")

cursor = conn.cursor() 

In [7]:
query = """
CREATE TABLE IF NOT EXISTS exchange_rates (
  exchange_id serial PRIMARY KEY,
  name VARCHAR(100) NOT NULL,
  unit VARCHAR(5) NOT NULL,
  value FLOAT, 
  type VARCHAR(20), 
  date TIMESTAMP  
);
"""

cursor.execute(query)
conn.commit()

In [ ]:
cursor.execute(f"""
                INSERT INTO customer (CustFName, CustLName, CustPhone, CustEmail, CustState, ContactPref, PmtID, AddedStamp,
                                  UpdatedStamp, HHI, IsMarried, HasKids, TravelsWPet, Pronoun)
                VALUES ({firstName}, {lastName}, {items[3]}, {items[4]}, {items[6]}, {items[5]}, {items[7]}, current_timestamp, 
                NULL, NULL, NULL, NULL, NULL, NULL)
                """)

    connPostgres.commit()